#### Imports

In [1]:
import os
from configs import ConfigPath
from utils.utils import read_json_file
from llms.embedding_model import EmbeddingModel
from configs.config import ConfigEnv
from knowledge_graph.connection import Neo4jConnection
from llms.llm import ChatModel
from retrieval.tools.vector_search_tool import VectorSearchTool
from answer_pip_executor import AnswerPipExecutor
from evaluation.evaluation import Evaluator

#### Initializations

In [2]:
# models
embedding_model = EmbeddingModel()

llm = ChatModel(provider="google", model_name="gemini-2.0-flash-lite").initialize_model()

# neo4j connection
neo4j_connection = Neo4jConnection(uri=ConfigEnv.NEO4J_URI, 
                 user=ConfigEnv.NEO4J_USER,
                 password=ConfigEnv.NEO4J_PASSWORD,
                 database=ConfigEnv.NEO4J_DB)

answer_type = "short"

# retriever
vector_search_tool = VectorSearchTool(
    llm=llm,
    embedding_model=embedding_model,
    neo4j_connection=neo4j_connection,
    return_direct=False,
    answer_type=answer_type,
)

retriever_model_name = vector_search_tool.get_model_name()

# data
data = read_json_file(file_path=os.path.join(ConfigPath.RAW_DATA_DIR, "test_pqa.json"))  
print(f"Data length: {len(data)}")

2025-03-17 00:27:57,395 [DEBUG] embedding_model - CUDA is available, using GPU
2025-03-17 00:28:54,606 [DEBUG] embedding_model - Embedding model initialized: neuml/pubmedbert-base-embeddings
2025-03-17 00:28:54,649 [DEBUG] llm - Initialized model gemini-2.0-flash-lite
2025-03-17 00:28:58,773 [DEBUG] connection - Connection successful!


Data length: 9


In [3]:
results = vector_search_tool.invoke("Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?")

In [3]:
answer_pip_executor = AnswerPipExecutor(source_data=data)
results = answer_pip_executor.generate_answers(retriever=vector_search_tool, model_name=retriever_model_name, answer_type=answer_type)

2025-03-17 00:29:00,579 [INFO] answer_pip_executor - Starting the answer generation...
Generating results...: 100%|██████████| 9/9 [00:08<00:00,  1.09it/s]
2025-03-17 00:29:08,869 [INFO] answer_pip_executor - Generated 9 results.


In [4]:
import pandas as pd

results_df = pd.DataFrame(results)

In [6]:
results_df["context_found"]

0    True
1    True
2    True
3    True
4    True
5    True
6    True
7    True
8    True
Name: context_found, dtype: bool

In [7]:
evaluator = Evaluator(ground_truth_data=data, short_answer_results=results)
evaluator.evaluate()

2025-03-16 23:44:36,174 [INFO] evaluation - Computing metrics...


KeyError: 'actual'

In [ ]:
evaluator.compute_short_answer_metrics()

2025-03-07 00:15:50,392 [INFO] evaluation - Computing metrics...
2025-03-07 00:15:50,508 [INFO] evaluation - Accuracy: 0.6780
2025-03-07 00:15:50,511 [INFO] evaluation - Precision: 0.6423
2025-03-07 00:15:50,512 [INFO] evaluation - Recall: 0.9402
2025-03-07 00:15:50,514 [INFO] evaluation - F1: 0.7632
2025-03-07 00:15:50,517 [INFO] evaluation - Results saved to C:\Users\melis\Desktop\Projects\python_projects\thesis\advanced_graph_rag\data\metrics\answering_metrics_20250307_001550.json


{'accuracy': 0.678,
 'precision': 0.6423267326732673,
 'recall': 0.9402173913043478,
 'f1': 0.763235294117647}

In [4]:
contexts = [element['content'] for element in results['context']]
contexts

['Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants. The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), cells in early stages of PCD (EPCD), and cells in late stages of PCD (LPCD). Window stage leaves were stained with the mitochondrial dye MitoTracker Red CMXRos 

In [5]:
from ragas import EvaluationDataset


dataset = []
dataset.append(
        {
            "user_input": "Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?",
            "retrieved_contexts": contexts,
            "response": "yes",
            "reference": "yes", # expected response
        }
    )
evaluation_dataset = EvaluationDataset.from_list(dataset)

In [15]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.metrics import ContextPrecision, ContextRecall, ResponseRelevancy, FactualCorrectness

evaluator_llm = LangchainLLMWrapper(llm)
evaluator_embedding = LangchainEmbeddingsWrapper(embedding_model)

context_precision = ContextPrecision()
context_recall = ContextRecall()
response_relevancy = ResponseRelevancy()
factual_correctness = FactualCorrectness()

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[context_precision, context_recall, response_relevancy, factual_correctness],
    llm=evaluator_llm,
    embeddings=evaluator_embedding
)

result

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

{'context_precision': 1.0000, 'context_recall': 1.0000, 'answer_relevancy': 0.0526, 'factual_correctness(mode=f1)': 1.0000}